In [ ]:
# Instalação das libs no Colab (execute 1 vez)
%apt-get update -qq
%apt install -y chromium-chromedriver
%cp /usr/lib/chromium-browser/chromedriver /usr/bin
%pip install requests beautifulsoup4 vaderSentiment matplotlib schedule twilio scikit-learn yfinance pandas
%pip install -q python-telegram-bot --upgrade


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 10.4 MB/s eta 0:00:00


In [ ]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import schedule
import time
import io
from telegram import Bot

# --- Configurações ---
CRIPTO_LISTA = ['bitcoin', 'ethereum', 'solana', 'aave', 'lido-dao', 'pendle']
ANALISADOR = SentimentIntensityAnalyzer()

# Chaves de API e Telegram Bot
NEWSAPI_KEY = "bfca953572834ef786d29a2869b07275"
TELEGRAM_TOKEN = "8123262775:AAHEv43aS9dK8jXSjINqhDXbqxlHAfn4aTw"
TELEGRAM_CHAT_ID = "7657570667"

bot = Bot(token=TELEGRAM_TOKEN)

# --- Funções ---
def buscar_noticias(cripto):
    query = f'{cripto} criptomoeda'
    url = f"https://news.google.com/search?q={query}&hl=pt-BR&gl=BR&ceid=BR%3Apt-419"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    titulos = [item.text for item in soup.select('article h3')]
    return titulos[:10]

def analisar_sentimento(noticias):
    scores = [ANALISADOR.polarity_scores(n)['compound'] for n in noticias]
    return sum(scores)/len(scores) if scores else 0

def obter_historico(cripto):
    url = f"https://api.coingecko.com/api/v3/coins/{cripto}/market_chart?vs_currency=usd&days=90"
    response = requests.get(url).json()
    prices = response['prices'] if 'prices' in response else []
    return pd.DataFrame(prices, columns=['timestamp', 'price']) if prices else pd.DataFrame()

def obter_dados_fundamentais(cripto):
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids={cripto}"
    response = requests.get(url).json()
    if not response:
        return {}, ""
    dados = response[0]
    texto = (
        f"🏦 Market Cap: ${dados['market_cap']:,.0f}\n"
        f"💸 Volume 24h: ${dados['total_volume']:,.0f}\n"
        f"🔄 Supply Circulante: {dados['circulating_supply']:,.0f}\n"
        f"🔢 Total Supply: {dados['total_supply'] if dados['total_supply'] else 'Não disponível'}"
    )
    return dados, texto

def calcular_rsi(df, periodo=14):
    delta = df['price'].diff()
    ganho = delta.where(delta > 0, 0.0)
    perda = -delta.where(delta < 0, 0.0)
    media_ganho = ganho.rolling(window=periodo).mean()
    media_perda = perda.rolling(window=periodo).mean()
    rs = media_ganho / media_perda
    return 100 - (100 / (1 + rs))

def calcular_bollinger(df, periodo=20):
    media = df['price'].rolling(window=periodo).mean()
    desvio = df['price'].rolling(window=periodo).std()
    banda_superior = media + (2 * desvio)
    banda_inferior = media - (2 * desvio)
    return banda_superior, banda_inferior

def treinar_lstm(df):
    dados = df['price'].values.reshape(-1,1)
    scaler = MinMaxScaler()
    dados_norm = scaler.fit_transform(dados)
    X, y = [], []
    for i in range(10, len(dados_norm)):
        X.append(dados_norm[i-10:i])
        y.append(dados_norm[i])
    X, y = np.array(X), np.array(y)
    model = Sequential([LSTM(50, return_sequences=False, input_shape=(10,1)), Dense(1)])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=10, verbose=0)
    ultima_seq = dados_norm[-10:].reshape(1,10,1)
    pred = model.predict(ultima_seq, verbose=0)
    return scaler.inverse_transform(pred)[0][0]

def clustering(df):
    dados = df['price'].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=3, random_state=42).fit(dados)
    return kmeans.labels_[-1]  # 0, 1 ou 2

def treinar_modelos_classificadores(df, sentimento):
    df['RSI'] = calcular_rsi(df)
    df = df.dropna()
    X = pd.DataFrame({
        'RSI': df['RSI'],
        'Sentimento': sentimento,
        'Preço': df['price']
    })
    X = X.fillna(method='ffill')
    y = X['RSI'].apply(lambda x: 1 if x < 30 else -1 if x > 70 else 0)
    modelos = {
        'DecisionTree': DecisionTreeClassifier(),
        'RandomForest': RandomForestClassifier(),
        'SVM': SVC()
    }
    resultados = {}
    for nome, modelo in modelos.items():
        modelo.fit(X, y)
        pred = modelo.predict([X.iloc[-1]])[0]
        resultados[nome] = pred
    return resultados

def gerar_grafico(df, nome):
    df['RSI'] = calcular_rsi(df)
    banda_sup, banda_inf = calcular_bollinger(df)
    fig, axs = plt.subplots(2, 1, figsize=(10,8), gridspec_kw={'height_ratios': [2, 1]})
    axs[0].plot(pd.to_datetime(df['timestamp'], unit='ms'), df['price'], label='Preço')
    axs[0].plot(pd.to_datetime(df['timestamp'], unit='ms'), banda_sup, label='Banda Superior', linestyle='--')
    axs[0].plot(pd.to_datetime(df['timestamp'], unit='ms'), banda_inf, label='Banda Inferior', linestyle='--')
    axs[0].set_title(f'{nome.upper()} - Preço e Bandas de Bollinger')
    axs[0].legend()
    axs[0].grid()
    axs[1].plot(pd.to_datetime(df['timestamp'], unit='ms'), df['RSI'], label='RSI', color='purple')
    axs[1].axhline(70, color='red', linestyle='--')
    axs[1].axhline(30, color='green', linestyle='--')
    axs[1].set_title('Índice de Força Relativa (RSI)')
    axs[1].legend()
    axs[1].grid()
    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    return buf

def salvar_alerta_csv(nome, preco, rsi, sentimento, previsao, cluster, timestamp):
    dados = {
        'Cripto': nome,
        'Preço': preco,
        'RSI': rsi,
        'Sentimento': sentimento,
        'Previsao_LSTM': previsao,
        'Cluster': cluster,
        'DataHora': timestamp
    }
    df = pd.DataFrame([dados])
    if not os.path.exists("alertas.csv"):
        df.to_csv("alertas.csv", index=False)
    else:
        df.to_csv("alertas.csv", mode='a', header=False, index=False)

def enviar_mensagem_telegram(msg, img_bytes):
    if img_bytes:
        bot.send_photo(chat_id=TELEGRAM_CHAT_ID, photo=img_bytes, caption=msg)
    else:
        bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=msg)

def analisar_criptomoeda(cripto):
    nome = cripto.upper()
    noticias = buscar_noticias(cripto)
    sentimento = analisar_sentimento(noticias)
    df = obter_historico(cripto)
    if df.empty or len(df) < 20:
        enviar_mensagem_telegram(f"{nome} - Dados insuficientes para análise.", None)
        return
    preco = df['price'].iloc[-1]
    rsi = calcular_rsi(df).iloc[-1]
    banda_sup, banda_inf = calcular_bollinger(df)
    dentro_banda = 'dentro' if banda_inf.iloc[-1] <= preco <= banda_sup.iloc[-1] else 'fora'
    pred_lstm = treinar_lstm(df)
    cluster = clustering(df)
    modelo_preds = treinar_modelos_classificadores(df, sentimento)
    dados_fund, texto_fund = obter_dados_fundamentais(cripto)

    rsi_msg = "RSI neutro"
    if rsi > 70:
        rsi_msg = "Possível Venda"
    elif rsi < 30:
        rsi_msg = "Possível Compra"

    bollinger_msg = f"Preço está {dentro_banda} das Bandas de Bollinger."
    cluster_msg = {
        0: "Mercado em fase de acumulação",
        1: "Mercado em alta volatilidade",
        2: "Mercado em tendência de alta"
    }.get(cluster, "Análise não classificada")

    ia_msg = (
        f"📈 Previsão LSTM: ${pred_lstm:,.2f}\n"
        f"🔍 Clustering: {cluster_msg}"
    )
    modelo_msg = "\n📉 Modelos Tradicionais:\n"
    for modelo, pred in modelo_preds.items():
        acao = {1: 'Compra', -1: 'Venda', 0: 'Manter'}.get(pred, 'Desconhecido')
        modelo_msg += f"🔹 {modelo}: {acao}\n"

    mensagem = (
        f"{nome} - Análise Completa\n"
        f"💰 Preço Atual: ${preco:,.2f}\n"
        f"🧠 Sentimento do Mercado: {'Positivo' if sentimento > 0 else 'Negativo' if sentimento < 0 else 'Neutro'} ({sentimento:.3f})\n"
        f"📊 RSI: {rsi_msg} ({rsi:.2f})\n"
        f"📈 Bandas de Bollinger: {bollinger_msg}\n"
        f"🤖 Análise IA:\n{ia_msg}\n"
        f"{modelo_msg}"
        f"\n📊 Dados Fundamentais:\n{texto_fund}\n"
        f"📅 {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}"
    )

    img = gerar_grafico(df, nome)
    salvar_alerta_csv(nome, preco, rsi, sentimento, pred_lstm, cluster, datetime.now())
    enviar_mensagem_telegram(mensagem, img)

def rodar_todas():
    for cripto in CRIPTO_LISTA:
        try:
            analisar_criptomoeda(cripto)
        except Exception as e:
            print(f"Erro ao analisar {cripto}: {e}")

schedule.every(3).hours.do(rodar_todas)

print("⏳ Análise de criptoativa agendada...")
rodar_todas()

while True:
    schedule.run_pending()
    time.sleep(1)


⏳ Análise de criptoativa agendada...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
<ipython-input-11-d3d7b3a7a576>:111: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid featu

KeyboardInterrupt: 